In [1]:
%tensorflow_version 2.x
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 2.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.1-cp36-none-any.whl size=30083 sha256=9a928fc205bd77aa4afdf47e3d8784b1adec97bcf577e2ee62ce3a6c90179a07
  Stored in directory: /root/.cache/pip/wheels/dd/f1/10/861fd7899727e4034293fb1dfef45b00f8cd476d21d3b3821e
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=6dc91f6ecd5fb11c14972a71d5db1f8b998e758eed268157375f66da8d00e03f
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.0-cp36-none-any.whl size=15999 sha256=b77e510c0f4bf3f63fcf0cd6815b82acd69029c23ae24f058df6d03732032437
  Stored in directory: /root/.cache/pip/wheels/88/41/05/1a9955d1d01575bbd58aab76e22f8c7eeabba905d551576f43
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.0MB 2.8MB/s 


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
from sklearn.model_selection import train_test_split
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from google.colab import files
import os
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="/content/gcs.json"

TensorFlow Version: 2.2.0-rc2
Hub version:  0.8.0


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Saving gcs.json to gcs.json
User uploaded file "gcs.json" with length 2297 bytes


In [3]:
!gcloud auth activate-service-account --key-file=/content/gcs.json
!gsutil cp  gs://my-data-bucket-colab/toxic-comments-classification/train.csv.zip train.csv.zip
#!gsutil cp gs://my-data-bucket-colab/toxic-comments-classification/test_labels.csv.zip test_labels.csv.zip 
#!gsutil cp gs://my-data-bucket-colab/toxic-comments-classification/test.csv.zip test.csv.zip 
!unzip train.csv.zip
#!unzip test.csv.zip

Activated service account credentials for: [gcs-40@pivotal001.iam.gserviceaccount.com]
Copying gs://my-data-bucket-colab/toxic-comments-classification/train.csv.zip...
- [1 files][ 26.3 MiB/ 26.3 MiB]                                                
Operation completed over 1 objects/26.3 MiB.                                     
Archive:  train.csv.zip
  inflating: train.csv               


In [0]:

def results(pred_results,ground_truth,threshold):
    pred_results = (pred_results>=threshold).astype(int)
    print("Positive labels RECALL-thre.{}:{}%".format(threshold,100*np.sum(pred_results*ground_truth)/np.sum(ground_truth)))
    cf = tfa.metrics.MultiLabelConfusionMatrix(num_classes=6)
    cf.update_state(ground_truth, pred_results)
    print('Confusion matrix:', cf.result().numpy())
    hl = tfa.metrics.HammingLoss(mode='multilabel', threshold=threshold)
    hl.update_state(ground_truth, pred_results)
    print('Hamming loss:', hl.result().numpy())


def index_set(k,pattern):
    m= 2**(5-k)
    return set([i for i in range(64) if (i//m%2)!=0]).intersection(set(pattern.keys()))


def label_to_class(v):
    w = np.array([32,16,8,4,2,1],dtype=np.int16)
    return np.sum(v*w,axis=1)

def class_to_label(c):
    c=np.reshape(c,(-1,1))
    v5=c%2
    c=c//2
    v4=c%2
    c=c//2
    v3=c%2
    c=c//2
    v2=c%2
    c=c//2
    v1=c%2
    c=c//2
    v0=c%2
    return np.concatenate([v0,v1,v2,v3,v4,v5],axis=1)
def distribution(pattern):
    b=class_to_label(np.array(list(pattern.keys())))
    a=np.asarray(list(pattern.values())).reshape(-1,1)
    M = a*b
    w_vec=np.sum(M,axis=0)
    #return w_vec/np.sum(w_vec)
    return w_vec

def class_count(s,pattern):
    S=0
    for k in list(s):
        S+=pattern[k]
    return S

In [0]:
# multi-label hamming loss
import tensorflow_addons as tfa
hl = tfa.metrics.HammingLoss(mode='multilabel', threshold=0.4)


In [0]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
MAX_SEQ_LEN=128
input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                    name="segment_ids")

pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
FullTokenizer=bert.bert_tokenization.FullTokenizer
vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()
tokenizer=FullTokenizer(vocab_file,do_lower_case)

In [0]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids
def create_single_input(sentence,MAX_LEN):
  stokens = tokenizer.tokenize(sentence)
  stokens = stokens[:MAX_LEN]
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)
  return ids,masks,segments
def create_input_array(sentences):
  input_ids, input_masks, input_segments = [], [], []
  for sentence in tqdm(sentences,position=0, leave=True):
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)
    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)
  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [8]:
@tf.function
def custom_recall(y_true,y_pred):
    y_true = tf.convert_to_tensor(y_true)
    Y_PRED = tf.cast(y_pred>=0.4,tf.float32)
    recall = tf.math.divide_no_nan(tf.math.reduce_sum(tf.math.multiply(y_true,Y_PRED)),tf.math.reduce_sum(y_true))
    return recall

x = tf.keras.layers.Dropout(0.1)(pooled_output)
x= tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
out = tf.keras.layers.Dense(6, activation="sigmoid", name="dense_output")(x)

model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[hl,custom_recall])
model.load_weights('gs://my-data-bucket-colab/toxic-comments-classification/chkpt')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [9]:
import pandas as pd
df=pd.read_csv('train.csv')
df = df.sample(frac=1)
train_sentences = df["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
inputs=create_input_array(train_sentences)

100%|██████████| 159571/159571 [03:13<00:00, 826.01it/s]


In [10]:
train_y = df[list_classes].values
features=np.concatenate(inputs,axis=1)
features=np.concatenate((features,train_y),axis=1)

_,features_test,_,_= train_test_split(features, train_y,test_size=0.25)
X = np.split(features_test,[128,2*128,3*128,390],axis=1)
input_ids_test=X[0]
input_masks_test=X[1]
input_segments_test=X[2]
test_y= X[3].astype(np.float32)

inputs_test=[input_ids_test,input_masks_test,input_segments_test]
print("Test set samples:",test_y.shape[0])

Test set samples: 39893


In [11]:
pred_results = model.predict(inputs_test,batch_size=256)
ground_truth = test_y.astype(int)
results(pred_results,ground_truth,0.4)
#results(pred_results,ground_truth,0.3)


Positive labels RECALL-thre.0.4:81.60220994475138%
Confusion matrix: [[[31789.  4166.]
  [  557.  3381.]]

 [[39040.   447.]
  [  161.   245.]]

 [[35301.  2419.]
  [  349.  1824.]]

 [[39548.   223.]
  [   58.    64.]]

 [[35381.  2455.]
  [  337.  1720.]]

 [[39266.   273.]
  [  203.   151.]]]
Hamming loss: 0.048663512
